<a href="https://colab.research.google.com/github/Vedmani/wear-particle-classification/blob/main/MobileNet.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [ ]:
!pip install wandb -qU
!wandb login 617d80dc70f383d8ff9ef1c2b94a78ccc8d8db23

     |████████████████████████████████| 1.9 MB 8.7 MB/s 
     |████████████████████████████████| 182 kB 66.1 MB/s 
     |████████████████████████████████| 162 kB 51.7 MB/s 
     |████████████████████████████████| 63 kB 1.9 MB/s 
     |████████████████████████████████| 162 kB 71.8 MB/s 
     |████████████████████████████████| 158 kB 68.9 MB/s 
     |████████████████████████████████| 157 kB 70.4 MB/s 
     |████████████████████████████████| 157 kB 56.5 MB/s 
     |████████████████████████████████| 157 kB 31.6 MB/s 
     |████████████████████████████████| 157 kB 50.0 MB/s 
     |████████████████████████████████| 157 kB 67.0 MB/s 
     |████████████████████████████████| 157 kB 69.8 MB/s 
     |████████████████████████████████| 157 kB 63.4 MB/s 
     |████████████████████████████████| 156 kB 75.8 MB/s 
wandb: Appending key for api.wandb.ai to your netrc file: /root/.netrc


In [ ]:
import tensorflow as tf
from tensorflow import keras
from tensorflow.keras import layers
from tensorflow.keras.models import Sequential
import wandb
from wandb.keras import WandbCallback

In [ ]:
tf.keras.utils.set_random_seed(1)
tf.config.experimental.enable_op_determinism()

In [ ]:
PATH = '/content/drive/MyDrive/Research Internship/final data/wear particles/Train'
test_path = '/content/drive/MyDrive/Research Internship/final data/wear particles/Test'
batch_size = 32
img_height = 224
img_width = 224

train_ds = tf.keras.utils.image_dataset_from_directory(
  PATH,
  validation_split=0.2,
  subset="training",
  seed=42,
  image_size=(img_height, img_width),
  batch_size=batch_size)

val_ds = tf.keras.utils.image_dataset_from_directory(
  PATH,
  validation_split=0.2,
  subset="validation",
  seed=42,
  image_size=(img_height, img_width),
  batch_size=batch_size)

test_ds = tf.keras.utils.image_dataset_from_directory(
  test_path,
  seed = 42,
  image_size=(img_height, img_width))

class_names = train_ds.class_names
test_class_names = test_ds.class_names
num_classes = len(class_names)
print(class_names)

Found 740 files belonging to 5 classes.
Using 592 files for training.
Found 740 files belonging to 5 classes.
Using 148 files for validation.
['cutting wear', 'normal rubbing and cutting wear combined', 'normal rubbing wear', 'red oxide', 'severe sliding wear']


In [ ]:
mobilenet = tf.keras.applications.MobileNet(
    input_shape=None,
    alpha=1.0,
    depth_multiplier=1,
    dropout=0.001,
    include_top=False,
    weights="imagenet",
    input_tensor=None,
    pooling=None,
    classes=1000,
    classifier_activation="softmax",
)
model_mobilenet=Sequential([
  layers.Rescaling(1./255, input_shape=(img_height, img_width, 3)),
  mobilenet,
  layers.Flatten(),
  layers.Dense(32, activation='relu'),
  layers.Dropout(0.2), 
  layers.Dense(num_classes, activation='softmax')
])
mobilenet.trainable=False
model_mobilenet.summary()

Model: "sequential_3"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 rescaling_1 (Rescaling)     (None, 224, 224, 3)       0         
                                                                 
 mobilenet_1.00_224 (Functio  (None, None, None, 1024)  3228864  
 nal)                                                            
                                                                 
 flatten_1 (Flatten)         (None, 50176)             0         
                                                                 
 dense_2 (Dense)             (None, 32)                1605664   
                                                                 
 dense_3 (Dense)             (None, 5)                 165       
                                                                 
Total params: 4,834,693
Trainable params: 4,812,805
Non-trainable params: 21,888
_______________________________________

In [ ]:
model_mobilenet.compile(
              optimizer= tf.keras.optimizers.Adam(learning_rate=0.001, beta_1=0.9, beta_2=0.999, epsilon=1e-07, amsgrad=False, name='Adam'),
              loss=tf.keras.losses.SparseCategoricalCrossentropy(from_logits=False),
              metrics=['accuracy']
              )

In [ ]:
wandb.init(
      # Set the project where this run will be logged
      project="Transfer_Learning_New", 
      name = "Mobilenet_base_trained",
)

ERROR:wandb.jupyter:Failed to detect the name of this notebook, you can set it manually with the WANDB_NOTEBOOK_NAME environment variable to enable code saving.
wandb: Currently logged in as: vedmanivaidya. Use `wandb login --relogin` to force relogin


In [ ]:
%%time
epochs=40
history = model_mobilenet.fit(
  train_ds,
  validation_data=val_ds,
  epochs=epochs,
callbacks=[WandbCallback()]
)

wandb: WARNING The save_model argument by default saves the model in the HDF5 format that cannot save custom objects like subclassed models and custom layers. This behavior will be deprecated in a future release in favor of the SavedModel format. Meanwhile, the HDF5 model is saved as W&B files and the SavedModel as W&B Artifacts.


Epoch 1/40
19/19 [==============================] - ETA: 0s - loss: 1.8247 - accuracy: 0.7280

wandb: Adding directory to artifact (/content/wandb/run-20221018_084524-2quqvkas/files/model-best)... Done. 0.6s


19/19 [==============================] - 229s 9s/step - loss: 1.8247 - accuracy: 0.7280 - val_loss: 2.4830 - val_accuracy: 0.4392
Epoch 2/40
19/19 [==============================] - 115s 6s/step - loss: 0.2735 - accuracy: 0.9257 - val_loss: 6.1671 - val_accuracy: 0.6014
Epoch 3/40
19/19 [==============================] - 111s 6s/step - loss: 0.0911 - accuracy: 0.9696 - val_loss: 5.1854 - val_accuracy: 0.5608
Epoch 4/40
19/19 [==============================] - ETA: 0s - loss: 0.2798 - accuracy: 0.9493

wandb: Adding directory to artifact (/content/wandb/run-20221018_084524-2quqvkas/files/model-best)... Done. 0.6s


19/19 [==============================] - 127s 7s/step - loss: 0.2798 - accuracy: 0.9493 - val_loss: 1.4993 - val_accuracy: 0.7905
Epoch 5/40
19/19 [==============================] - ETA: 0s - loss: 0.1099 - accuracy: 0.9645

wandb: Adding directory to artifact (/content/wandb/run-20221018_084524-2quqvkas/files/model-best)... Done. 0.8s


19/19 [==============================] - 133s 7s/step - loss: 0.1099 - accuracy: 0.9645 - val_loss: 0.1212 - val_accuracy: 0.9595
Epoch 6/40
19/19 [==============================] - 113s 6s/step - loss: 0.1275 - accuracy: 0.9696 - val_loss: 0.2326 - val_accuracy: 0.9730
Epoch 7/40
19/19 [==============================] - 114s 6s/step - loss: 0.0479 - accuracy: 0.9882 - val_loss: 0.2029 - val_accuracy: 0.9595
Epoch 8/40
19/19 [==============================] - 113s 6s/step - loss: 0.0683 - accuracy: 0.9899 - val_loss: 0.2697 - val_accuracy: 0.9662
Epoch 9/40
19/19 [==============================] - 115s 6s/step - loss: 0.0932 - accuracy: 0.9780 - val_loss: 0.2442 - val_accuracy: 0.9662
Epoch 10/40
19/19 [==============================] - 113s 6s/step - loss: 0.0138 - accuracy: 0.9966 - val_loss: 0.3361 - val_accuracy: 0.9730
Epoch 11/40
19/19 [==============================] - 114s 6s/step - loss: 0.0202 - accuracy: 0.9932 - val_loss: 0.1956 - val_accuracy: 0.9730
Epoch 12/40
19/19 [===

KeyboardInterrupt: ignored

In [ ]:
wandb.finish()

accuracy,▁▆▇▇▇▇██▇███████
epoch,▁▁▂▂▃▃▄▄▅▅▆▆▇▇██
loss,█▂▁▂▁▁▁▁▁▁▁▁▁▁▁▁
val_accuracy,▁▃▃▅████████████
val_loss,▄█▇▃▁▁▁▁▁▁▁▁▁▁▁▁
accuracy,1.0
best_epoch,4
best_val_loss,0.12121
epoch,15
loss,0.00153
val_accuracy,0.97297
